# *The missing link:*
# CAS Applied Data Science Final Project
# Matthias Rinderknecht

## Notebook 1: Dataset preparation and cleaning

In [ ]:
import pandas as pd
import re
import tkinter as tk
from tkinter import filedialog
from bs4 import BeautifulSoup

# Dataset preparation

## ICTRP-only dataset

### Print statistics

In [5]:
# Read the CSV file into a DataFrame
input_file = "/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/export_snctp-2023-11-14_new.csv"
df = pd.read_csv(input_file)

# Convert all columns to strings
df = df.astype(str)

# Print an overview of the cleaned DataFrame
print("Shape of the DataFrame:", df.shape)
print("\nMissing Data per Column (string == 'nan'):")
for col in df.select_dtypes(include=['object']):
    missing_count = (df[col].str.lower() == 'nan').sum()
    print(f"{col}: {missing_count}")
print("\nData types of columns:")
print(df.dtypes)

print("\nSummary statistics:")
pd.set_option('display.max_columns', None)
print(df.describe())



/var/folders/3f/fq1cp83n5sq16c_d53znhsfh0000gn/T/ipykernel_1251/2170802494.py:6: DtypeWarning: Columns (1,5,45,46,47,48,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Shape of the DataFrame: (64209, 64)

Missing Data per Column (string == 'nan'):
date: 0
basecId: 60162
snctpId: 59271
whoId: 60097
ECName: 59394
ecFinalDecisionDate: 60169
WHO register: 60215
flagRareDisease: 59271
flagForChildren: 59271
flagForAdolescents: 59271
flagForHealthy: 59271
published: 59271
contactName: 59294
contactMail: 59322
contactPhone: 59299
lang: 59271
layTitle: 59273
laySummary: 59274
disease: 59282
intervention: 59285
inclusionCriteria: 59274
exclusionCriteria: 59274
studySitesOther: 63556
countries: 1908
studysites: 59552
tags: 59351
trialId: 1822
url: 1823
publicTitle: 1908
recruitmentStatus: 2207
secondaryId: 3556
scientificTitle: 2677
inclusionCriteria.1: 2522
exclusionCriteria.1: 31819
interventions: 5164
dateEnrollement: 3923
dateRegistration: 2995
studyType: 1842
studyDesign: 7678
primarySponsor: 1864
secondarySponsors: 55567
phase: 29126
healthConditions: 2710
primaryOutcome: 3641
secondaryOutcomes: 13788
resultsSummary: 57910
resultsUrlLink: 58759
resultsIp

### Keep only ICTRP columns, and filter for country = Switzerland

In [48]:
# Read the CSV file into a DataFrame
input_file = "/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/export_snctp-2023-11-14_new.csv"
df = pd.read_csv(input_file)

# Convert all columns to strings
df = df.astype(str)

# keep only certain _ictrp columns 
df = df[["trialId", "scientificTitle", "publicTitle", "interventions", "healthConditions", "countries", "secondaryId", 
         "inclusionCriteria.1", "exclusionCriteria.1", "studyType", "studyDesign", "primarySponsor", 
         "secondarySponsors", "phase", "primaryOutcome", "sourceSupport", "alternativeNames", 
         "publicContactLastname", "publicContactEmail", "publicContactAffiliation", "scientificContactLastname",
         "scientificContactEmail", "scientificContactAffiliation"]]

# Keeps only rows where 'country' column contains "Switzerland"
df = df[df['countries'].str.contains('Switzerland', case=False)]

# Print an overview of the cleaned DataFrame
print("Shape of the DataFrame:", df.shape)
print("\nMissing Data per Column (string == 'nan'):")
for col in df.select_dtypes(include=['object']):
    missing_count = (df[col].str.lower() == 'nan').sum()
    print(f"{col}: {missing_count}")
print("\nData types of columns:")
print(df.dtypes)

print("\nSummary statistics:")
pd.set_option('display.max_columns', None)
print(df.describe())

/var/folders/3f/fq1cp83n5sq16c_d53znhsfh0000gn/T/ipykernel_1251/2512461373.py:6: DtypeWarning: Columns (1,5,45,46,47,48,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Shape of the DataFrame: (12820, 23)

Missing Data per Column (string == 'nan'):
trialId: 0
scientificTitle: 226
publicTitle: 6
interventions: 876
healthConditions: 141
countries: 0
secondaryId: 435
inclusionCriteria.1: 119
exclusionCriteria.1: 8872
studyType: 0
studyDesign: 1753
primarySponsor: 6
secondarySponsors: 10009
phase: 4910
primaryOutcome: 499
sourceSupport: 380
alternativeNames: 10954
publicContactLastname: 3638
publicContactEmail: 7495
publicContactAffiliation: 2291
scientificContactLastname: 3248
scientificContactEmail: 7328
scientificContactAffiliation: 2250

Data types of columns:
trialId                         object
scientificTitle                 object
publicTitle                     object
interventions                   object
healthConditions                object
countries                       object
secondaryId                     object
inclusionCriteria.1             object
exclusionCriteria.1             object
studyType                       object
studyDes

In [49]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/ICTRP_only(CH=true)_12820x23.csv', index=False)

## BASEC+ICTRP dataset

### Remove all rows not linked between BASEC and ICTRP, drop some more specific rows

In [1]:
# Read the CSV file into a DataFrame
input_file = "/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/export_snctp-2023-11-14_new.csv"
df = pd.read_csv(input_file)

#drop all rows with empty snctpId or empty layTitle
df = df.dropna(subset=['snctpId'])
df = df.dropna(subset=['layTitle'])

# Remove duplicates based on the "snctpId" column
df.drop_duplicates(subset='snctpId', inplace=True)
df = df.drop_duplicates(subset=['layTitle'], keep=False)

# Convert all columns to strings
df = df.astype(str)

# Drop rows where "trialId" is not equal to "whoId", not correctly linked trials
df = df[df['trialId'] == df['whoId']]

# Drop some specific test trials
df = df[~df['snctpId'].str.contains('000002335', na=False)]
df = df[~df['snctpId'].str.contains('000002095', na=False)]
df = df[~df['snctpId'].str.contains('000001884', na=False)]
df = df[~df['snctpId'].str.contains('000001897', na=False)]
df = df[~df['snctpId'].str.contains('000001831', na=False)]
df = df[~df['snctpId'].str.contains('000001952', na=False)]
df = df[~df['snctpId'].str.contains('000001993', na=False)]
df = df[~df['whoId'].str.contains('NCT03643276', na=False)]
df = df[~df['whoId'].str.contains('NCT05196035', na=False)]

# Print an overview of the cleaned DataFrame
print("Shape of the DataFrame:", df.shape)
print("\nMissing Data per Column (string == 'nan'):")
for col in df.select_dtypes(include=['object']):
    missing_count = (df[col].str.lower() == 'nan').sum()
    print(f"{col}: {missing_count}")
print("\nData types of columns:")
print(df.dtypes)

print("\nSummary statistics:")
print(df.describe())



Shape of the DataFrame: (3054, 64)

Missing Data per Column (string == 'nan'):
date: 0
basecId: 826
snctpId: 0
whoId: 0
ECName: 110
ecFinalDecisionDate: 830
WHO register: 108
flagRareDisease: 0
flagForChildren: 0
flagForAdolescents: 0
flagForHealthy: 0
published: 0
contactName: 21
contactMail: 45
contactPhone: 25
lang: 0
layTitle: 0
laySummary: 1
disease: 5
intervention: 7
inclusionCriteria: 0
exclusionCriteria: 0
studySitesOther: 2625
countries: 63
studysites: 161
tags: 25
trialId: 0
url: 0
publicTitle: 0
recruitmentStatus: 10
secondaryId: 77
scientificTitle: 28
inclusionCriteria.1: 63
exclusionCriteria.1: 2618
interventions: 132
dateEnrollement: 588
dateRegistration: 472
studyType: 1
studyDesign: 206
primarySponsor: 0
secondarySponsors: 2176
phase: 1392
healthConditions: 64
primaryOutcome: 68
secondaryOutcomes: 383
resultsSummary: 2857
resultsUrlLink: 2551
resultsIpdPlan: 2366
resultsIpdDescription: 2844
sourceSupport: 65
alternativeNames: 2869
dateCompletion: 2789
publicContactFirst

In [2]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/BASEC_with_ICTRP_3054x64.csv', index=False)

### Specify columns for _basec and _ictrp subsets

In [132]:
# Specify the columns to keep
# for basec: 
#columns_to_keep = ['basecId', 'snctpId', 'whoId', "layTitle", "laySummary", "disease", "intervention", "inclusionCriteria", "exclusionCriteria", "studysites", "studySitesOther", "tags"]
# for ictrp:
columns_to_keep = ["trialId", "publicTitle", "secondaryId",  "scientificTitle", "inclusionCriteria", "exclusionCriteria", "interventions", "primarySponsor", "healthConditions", "publicContactAffiliation", "scientificContactAffiliation"]
df = df[columns_to_keep]

# Print an overview of the cleaned DataFrame
print("Shape of the DataFrame:", df.shape)
print("\nData Types:")
print(df.dtypes)
print("\nMissing Data per Column (string == 'nan'):")
for col in df.select_dtypes(include=['object']):
    missing_count = (df[col].str.lower() == 'nan').sum()
    print(f"{col}: {missing_count}")
print("\nDescriptive Statistics:")
print(df.describe())

Shape of the DataFrame: (3054, 11)

Data Types:
trialId                         object
publicTitle                     object
secondaryId                     object
scientificTitle                 object
inclusionCriteria               object
exclusionCriteria               object
interventions                   object
primarySponsor                  object
healthConditions                object
publicContactAffiliation        object
scientificContactAffiliation    object
dtype: object

Missing Data per Column (string == 'nan'):
trialId: 0
publicTitle: 0
secondaryId: 77
scientificTitle: 28
inclusionCriteria: 0
exclusionCriteria: 0
interventions: 132
primarySponsor: 0
healthConditions: 64
publicContactAffiliation: 555
scientificContactAffiliation: 621

Descriptive Statistics:
            trialId                                        publicTitle  \
count          3054                                               3054   
unique         3026                                               

In [133]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/BASEC_with_ICTRP_3054x11_ictrp.csv', index=False)

# BASEC_without_ICTRP dataset 
check for twins and remove all if at least one copy has a trialId, but keep one copy if no duplicate has a trialId. then remove all other trials that have a trialId and layTitle duplicates and empty layTitle rows

In [122]:
# Read the CSV file
input_file = "/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/BASEC_with_or_without_ICTRP_4938x64.csv"
df = pd.read_csv(input_file)

# Print the number of rows in the CSV
original_rows = len(df)
print("Number of rows in the CSV:", original_rows)

# Find duplicates based on 'snctpId' column and 'trialId' column
duplicate_rows = df[df.duplicated(subset=['snctpId'], keep=False)]

# Filter duplicates where at least one has a non-null entry in 'trialId' column
non_null_trial_duplicates = duplicate_rows[duplicate_rows['trialId'].notnull()]

# Filter duplicates where all have null entry in 'trialId' column
null_trial_duplicates = duplicate_rows[duplicate_rows['trialId'].isnull()]

# Keep only one copy for duplicates where all have null entry in 'trialId' column
null_trial_duplicates = null_trial_duplicates.drop_duplicates(subset=['snctpId'])

# Concatenate both types of duplicates
all_duplicate_rows = pd.concat([non_null_trial_duplicates, null_trial_duplicates])

# Save dropped duplicate rows to a separate CSV file
dropped_rows_filename = "/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/dropped_rows.csv"
all_duplicate_rows.to_csv(dropped_rows_filename, index=False)

# Drop duplicates after filtering
df = df.drop_duplicates(subset=['snctpId'], keep=False)

# Drop rows where 'trialId' column is null
df = df[df['trialId'].isnull()]

# Drop layTitle duplicates
df = df.drop_duplicates(subset=['layTitle'], keep=False)

# Calculate the number of rows dropped after dropping non-NULL trialId rows
rows_dropped_trialId_not_null = original_rows - len(all_duplicate_rows) - len(df)

# Print the number of duplicate rows dropped, the number of non-NULL trialId rows dropped, and the number of rows remaining
print("Number of duplicate rows dropped:", len(all_duplicate_rows))
print("Number of non-NULL trialId rows dropped:", rows_dropped_trialId_not_null)
print("Number of rows remaining:", len(df))


Number of rows in the CSV: 4938
Number of duplicate rows dropped: 553
Number of non-NULL trialId rows dropped: 2849
Number of rows remaining: 1536


In [126]:
# Specify the columns to keep
# for basec: 
columns_to_keep = ['basecId', 'snctpId', 'whoId', "layTitle", "laySummary", "disease", "intervention", "inclusionCriteria", "exclusionCriteria", "studysites", "studySitesOther", "tags"]
df = df[columns_to_keep]

# Convert all columns to strings
df = df.astype(str)

df = df[df['layTitle'].str.lower() != 'nan']

# Print an overview of the cleaned DataFrame
print("Shape of the DataFrame:", df.shape)
print("\nData Types:")
print(df.dtypes)
print("\nMissing Data per Column (string == 'nan'):")
for col in df.select_dtypes(include=['object']):
    missing_count = (df[col].str.lower() == 'nan').sum()
    print(f"{col}: {missing_count}")
print("\nDescriptive Statistics:")
print(df.describe())

Shape of the DataFrame: (1535, 12)

Data Types:
basecId              object
snctpId              object
whoId                object
layTitle             object
laySummary           object
disease              object
intervention         object
inclusionCriteria    object
exclusionCriteria    object
studysites           object
studySitesOther      object
tags                 object
dtype: object

Missing Data per Column (string == 'nan'):
basecId: 45
snctpId: 0
whoId: 825
layTitle: 0
laySummary: 0
disease: 4
intervention: 3
inclusionCriteria: 1
exclusionCriteria: 1
studysites: 103
studySitesOther: 1348
tags: 52

Descriptive Statistics:
       basecId         snctpId whoId  \
count     1535            1535  1535   
unique    1491            1535   647   
top        nan  SNCTP000000470   nan   
freq        45               1   825   

                                                 layTitle  \
count                                                1535   
unique                            

In [129]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('/Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/BASEC_without_ICTRP_1535x12.csv', index=False)

# Data cleaning

### BASEC subsets

In [ ]:
# BASEC: copy layTitle into new column "layTitleCorr", convert to strings and lowercase
df1['layTitleCorr'] = df1['layTitle'].copy()
df1['layTitleCorr'] = df1['layTitleCorr'].astype(str)
df1['layTitleCorr'] = df1.apply(lambda row: row['layTitleCorr'].lower() if row['layTitleCorr'].isupper() else row['layTitleCorr'], axis=1)

### ICTRP subsets

In [1]:
"""With this version, the script iterates over all columns in the DataFrame and replaces <br> with a 
whitespace in each column except the "healthConditions" column. The "healthConditions" column is cleaned 
using the clean_text() function as before.
"""

# Define the clean_text() function
def clean_text(text):
    # Remove HTML tags
    cleaned_text = re.sub(r'<.*?>', '', text)
    # Remove alphanumeric codes like M50.1, T50, T20-35, P205.20
    cleaned_text = re.sub(r'\b[A-Z]\d{1,3}(-\d{1,3})?(\.\d{1,3})?\b', '', cleaned_text)
    # Remove any additional characters
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', cleaned_text)
    return cleaned_text.strip()  # Remove leading and trailing whitespaces

# Define a function to handle the cleaning process
def clean_csv(file_path):
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(file_path)
    df['scientificTitleCorr'] = df['scientificTitle'].fillna(df['publicTitle']).str.lower()
    df['publicTitleCorr'] = df['publicTitle'].copy().str.lower()
    # Convert all columns to strings
    df = df.astype(str)
    
    # Clean the data in the 'healthConditions' column
    df['healthConditions'] = df['healthConditions'].apply(clean_text)
    # Replace '<br>' with a whitespace in all other columns
    for column in df.columns:
        if column != 'healthConditions':
            df[column] = df[column].str.replace('<br>', ' ')
    # Save the cleaned DataFrame to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned_all.csv')
    df.to_csv(cleaned_file_path, index=False)
    print(f"File cleaned and saved as: {cleaned_file_path}")

# Open a file dialog to choose a CSV file
file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
if file_path:
    # Call the clean_csv() function when a file is selected
    clean_csv(file_path)


File cleaned and saved as: /Users/marinder/Documents/CAS_ADS/Final Project/Source Data/final_source/csv/BASEC_with_ICTRP_3054x11_ictrp_cleaned_all.csv
